<a href="https://colab.research.google.com/github/YakovDeLeon/ML_Course_PT/blob/main/%22PT_HT3_WhoIsTalking_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 3

В этом задании мы предлагаем вам не только улучшить [бейзлайн, который мы дали вам на занятии](https://colab.research.google.com/drive/1DirkE2idWXfl7TqTF0ZxBi6M37LZ0NGD?usp=sharing), но и продумать задание с точки зрения ML System Design. Ответы на вопросы, которые мы поставим перед вами, могут повлиять на код (метрики, модели и так далее).

Вам предстоит спроектировать систему классификации TCP-потоков по приложению/сервису на основе усечённой последовательности длин пакетов (до 30 значений, с направлением).

Представьте, что вы не просто обучаете модель, а проектируете промышленное решение, которое будет использоваться в корпоративной среде.

### Важно!

Сначала почитайте и изучите ноутбук и код с занятия.

# 1. Бизнес-контекст и целеполагание

Вопрос:
Зачем и кому может быть нужна система классификации приложений по зашифрованному трафику? Как и где она будет использоваться?

В ответе опишите:

- Кто является заказчиком (SOC-команда, отдел ИБ, провайдер, корпорация, государственная структура и т.д.)?

-  Какие реальные бизнес-задачи она решает?

- В каком режиме работает система:

офлайн-анализ (батч),

near-real-time,

real-time (inline)?

- Какие требования к:

задержке (latency),

пропускной способности,

интерпретируемости,

объяснимости,

обновляемости модели?

- Какие риски есть при ошибках классификации (FP/FN)?


# Ответ

Классификация приложений по зашифрованному трафику нужна в первую очередь для систем съёма трафика, в которых необходимо определить какие сервисы посещает пользователь и ограничивать доступ к нежелательному контенту со стороны системного администратора и блокировать различные кибер угрозы. Системы съёма трафика могут быть как активными, так и пассивными. Активные системы, например файрволл, встраиваются по середине между отправителем и получателем и могут модфифицировать оригинальные пакеты и блокировать трафик в момент его прихода, поэтому относятся к системам жёсткого реального времени. При пассивном съёме, например DPI, в систему попадает копия уже запроксированного трафика. Такие системы используются в основном для анализа трафика и выявления в нём различных аномалий. Так как в такую систему попадает уже копия трафика требования к задержкам тут ниже, но всё же хотелось выявить атаку как можно быстрее. Поэтому такие системы относятся к системам мягкого реального времени.

Для определённости, в рамках текущей задачи попытаемся встроить классификацию приложений в зашифрованном траффике в файрвол. Файрвол обычно поддерживает MITM и умеет дешифровать трафик, однако весь трафик обычно не дешифруются, так как это требует огромных вычислительных ресурсов, поэтому через файрвол TLS идёт в соотношении: 30% - расшифровывается, 70% - нет. Зашифрованный трафик файрвол уже умеет классифицировать по SNI, который передаётся при установлении соединения. Однако злоумышленник может использовать фейковый SNI и тем самым обходить политики безопасности.

Исходя из этого требования к модели следующие:
* Анализ в режиме real-time, любые задержки могут быть критичные для пользователя
* Latency: Модель должна принимать решение не позднее чем через 4кб для каждой из сторон, или суммарно 8 кб.
* В иделае модель должна быть легковесной и не должна влиять на пропускную способность, как будет на практике сказать сложно, но не хотелось допустить сильную деградацию производительности
* Интерпретируемость: Решение модели должно быть трассируемым до набора входных признаков.
* Модель должна содержать информацию о принятом решении, чтобы SOC команда смогла отличить правильное решение модели от ложного срабатывания
* Модель должна обновляться ежедневно, чтобы оперативно реагировать на новые угрозы. Обновление модели не должно приводить к перезапуску NGFW и влиять на обработку DP данных
* При ложном срабатывании может блокироваться легитимный трафик, поэтому должна быть надстройка на СУ, позволяющая отключить учёт результата модели



# 2. Данные и обработка: классы, дисбаланс, новые сервисы

Вопрос:
- Как нужно подготовить данные и организовать работу с классами, учитывая сильный дисбаланс?

В ответе раскройте:

- Нужно ли:

агрегировать редкие классы в "Other"?

удалять самые редкие классы?


- Как дисбаланс повлияет на:

обучение,

переобучение,

выбор метрики,

интерпретацию результатов?


- Какие стратегии борьбы с дисбалансом вы бы применили?

- Как система должна реагировать на новые приложения, которых не было в обучающей выборке?

- Нужно ли учитывать изменение поведения сервисов со временем (concept drift)?

# Ответ:

Для ngfw 20% от известных приложений будет составлять примерно 80% всего трафика. На остальные 70% приложений придётся, проценты или доли процентов трафика и ещё 10% попадут под нулевые срабатывания.

Поэтому очень редкие классы будет уместно аггрегировать в какую-то более общую категорию. Те классы, которые не представляют угрозы и имеют мало примеров трафика имеет смысл объединить в "Unknown application". Классы, которые являются редкими, но критичными к безопасности (например VPN, TOR, C2) нужно сохранять.

Самые редкие классы имееть смысл удалять, если:
* класс является шумовым
* плохо размечен
* имеет слишком мало данных
* не является значимым для security

В идеале такие классы лучше агрегировать в один или переводить в "Unknown applications".

Посколько самые популярные классы всегда будут составлять большую долю трафика возникает дисбаланс. Поэтому при обучении важно вносить корректировки в модель, иначе модель:
* будет минимизировать общую ошибку
* будет игнорировать миноритарные классы

Пример:

YouTube — 60%
Slack — 20%
Zoom — 10%
VPN — 0.5%
TOR — 0.1%

Модель может полностью игнорировать TOR и всё равно иметь 99% accuracy.

При таком дисбалансе, существует большой риск, что миноритарные классы:
* легко переобучаются
* модель просто запоминает конкретные TLS-потоки
* плохая генерализация (плохая обобщающая способность)

При выборре метрики общая accuracy будет работать плохо, так как мы не будем видеть какие классы путаем между собой. Например ютуб будет угадываться всегда, VPN - нет, но accuracy будет высокий. Поэтому в этом случае лушче исопльзовать:
* матрицу предсказаний (precision, recall)
* f1-меру
* каждую из этих метрик смотрим по сработке на каждый конкретный класс

Для борбы с дисбалансом будем использовать взвешивание классов. Для более редких будем назначать более высокий способ. Самый простой способ в качестве веса взять 1 / частоту встречания (других пока не знаю:-)). Однако надо понимать, что на разных предприятих, где установлен NGFW, может быть разный трафик, поэтому здесь хорошо бы собрать данных с разных мест.

Чатгпт ещё предлагает попробовать иерархическую классификацию, но это может увеличить сложность вычислений и привести к большей деградации, поскольку несколько моделей надо запускать по несколько раз на одном и том же наборе данных. Это может быть особенно заметно, если мы захотим также гонять содержимое зашифрованного трафика (хотя пока сложно сказать, насколько в этом есть смысл). Точно сказать, как себя поведёт модель тут сложно, надо исследовать.

При появлении новых приложений, модель может относить их к уже известным классам, что может быть не очень хорошо, так как новые VPN будут классифицироваться например как YouTube. Поэтому для контроля для с этим чатгпт советует:
* Threshold по confidence. Если max probability < T → Unknown
* Energy-based detection
* Distance to class centroid
* One-class classifier поверх эмбеддингов

Моделью должен обязательно учитываться concept-drift. Так как большинство приложений используют в основном REST API поверх HTTP внутри TLS, то взаимодействие между клеинтом и сервером может меняться со времени из-за обновления протокола взаимодействия. Это может отразиться на разных размерах пакетов, на разных урлах внутри процедуры обмена сообщения, если в каждом регионе размещён свой сервер.

Исходя из этого drift может быть:
* Feature drift (изменились количество данных отправялемых клиентом и сервером за тот же промежуток наблюдения)
* Prior drift (изменилось распределение классов, например из-за замедления ютуба, в топ может вылезти рутуб)
* Label drift (класс изменил поведение, (сомнительно, но окэй, не придумал пример такого))

Что с этим делать:
* Continuous monitoring
  * Track per-class recall over time
  * Track feature distribution
* Периодическое переобучение
  * Раз в день/месяц
  * Или при детекте деградации
  * Но тут вопрос какие данны подавать для переобчения, вряд ли снимать весь трафик на терабайты и подавать его на переобучение будет хорошей идеей. Наверное тут правильнее будет отдавать трафик с данными, которые попали под категорию "Unknown Applictaion"

* Shadow validation
  * Новая модель проверяется параллельно старой.


# 3. Метрика и выбор алгоритмов

Вопрос:
Какую метрику качества вы выберете и почему? Нужно ли ограничивать себя определёнными алгоритмами?


- Выберите и обоснуйте метрику.

- Объясните, почему выбранная метрика соответствует бизнес-целям.

- Нужно ли использовать несколько метрик?

- Допустимы ли любые модели? Есть ли ограничения?

# Ваш ответ здесь!

Как уже было сказано ранее, у нас имеется сильный дисбаланс классов, при этом NGFW должен не пропускать угрозы (то есть должен быть высокий recall) и не блокировать легитимный трафик (должен быть высокий precision). F1 является компроммисом между ними. Но для информативности будем использовать все три метрики:
* Для оценки безопасности, используем recall, так как среди всего трафика мы должны блокировать как можно больше того, что попадает под угрозы
* Для оценки стабильности используем Precision, чтобы понимать насколько можно доверять результату
* Для оценки общей устойчивости, будем использовать f-score

Модель не должна оказывать сильного влияния на обработку сетевого трафика. Поэтому ма должны ориентироваться на максимально лёгкие модели, например:
* Logistic Regression
* Gradient Boosted Trees (ограниченной глубины)


# 4. Персональные данные, приватность и деплой

Вопрос:
Считается ли такой датасет персональными данными? Какие риски существуют?


- Можно ли по последовательности размеров пакетов деанонимизировать пользователя?

- Где может быть развернута система:

on-premise,

в облаке,

на edge-устройствах?

- Какие меры нужно предусмотреть:

анонимизация,

логирование,

ограничение хранения,

контроль доступа?

# Ваш ответ здесь!

Да такой датасет является персональными данными. В худшем случае если соотнести размеры пакетов с трафиком, где они были они добыты, то можно узнать ip пользователя. По нему найти весь его трафик, а там если много вычислительных ресурсов можно и весь трафик попытаться дешифровать.

Система должна быть развёрнута on-permise. Для обучения модели трафик можно сгенерировать своими силами внутри компании, поэтому нет смысла выносить систему куда-то ещё.

Так как модель не покидает периметра компании, то кажется кроме контроля доступа здесь дополнительно ничего не треубется.

# Улучшение бейзлайна.

Наконец, вы можете писать код!

С учетом выбранной метрики и выбранного пула алгоритмов, улучшите бейзлайн с занятия:

- Поработайте с признаками - придумайте новые, затем попробуйте отбор признаков
- Поработайте с разными моделями, подбирайте у них гиперпараметры
- Можете попробовать использовать кластеризацию для извлечения дополнительных признаков

In [ ]:
# Ваш код здесь!